In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import plotly.express as px

In [2]:
temperature_data = pd.read_csv('temperature.csv')
co2_data = pd.read_csv('carbon_emmission.csv')

temperature_data.head(),co2_data.head()

(   ObjectId                       Country ISO2 ISO3  F1961  F1962  F1963  \
 0         1  Afghanistan, Islamic Rep. of   AF  AFG -0.113 -0.164  0.847   
 1         2                       Albania   AL  ALB  0.627  0.326  0.075   
 2         3                       Algeria   DZ  DZA  0.164  0.114  0.077   
 3         4                American Samoa   AS  ASM  0.079 -0.042  0.169   
 4         5      Andorra, Principality of   AD  AND  0.736  0.112 -0.752   
 
    F1964  F1965  F1966  ...  F2013  F2014  F2015  F2016  F2017  F2018  F2019  \
 0 -0.764 -0.244  0.226  ...  1.281  0.456  1.093  1.555  1.540  1.544  0.910   
 1 -0.166 -0.388  0.559  ...  1.333  1.198  1.569  1.464  1.121  2.028  1.675   
 2  0.250 -0.100  0.433  ...  1.192  1.690  1.121  1.757  1.512  1.210  1.115   
 3 -0.140 -0.562  0.181  ...  1.257  1.170  1.009  1.539  1.435  1.189  1.539   
 4  0.308 -0.490  0.415  ...  0.831  1.946  1.690  1.990  1.925  1.919  1.964   
 
    F2020  F2021  F2022  
 0  0.498  1.327  2.01

In [3]:
temperature_values = temperature_data.filter(regex='^F').stack()
temperature_stats = {
    "Mean": temperature_values.mean(),
    "Median": temperature_values.median(),
    "Variance": temperature_values.var()
}

co2_values = co2_data["Value"]
co2_stats = {
    "Mean": co2_values.mean(),
    "Median": co2_values.median(),
    "Variance": co2_values.var()
}

temperature_stats, co2_stats

({'Mean': np.float64(0.5377713483146068),
  'Median': 0.47,
  'Variance': 0.4294524831504378},
 {'Mean': np.float64(180.71615286624203),
  'Median': 313.835,
  'Variance': 32600.00200469294})

In [4]:
temperature_years = temperature_data.filter(regex='^F').mean(axis=0)
temperature_years.index = temperature_years.index.str.replace('F', '').astype(int)

co2_data['Year'] = co2_data['Date'].str[:4].astype(int)
co2_yearly = co2_data.groupby('Year')['Value'].mean()

fig = go.Figure()
fig.add_trace(go.Scatter(
    x=temperature_years.index, y=temperature_years.values,
    mode='lines+markers', name="Temperature Change (°C)"
))
fig.add_trace(go.Scatter(
    x=co2_yearly.index, y=co2_yearly.values,
    mode='lines+markers', name="CO₂ Concentration (ppm)", line=dict(dash='dash')
))
fig.update_layout(
    title="Time-series of Temperature Change and CO₂ Concentrations",
    xaxis_title="Year",
    yaxis_title="Values",
    template="plotly_white",
    legend_title="Metrics"
)
fig.show()

In [5]:
merged_data = pd.DataFrame({"Temperature Change": temperature_years,"CO₂ Concentration": co2_yearly}).dropna()
merged_data.head()

,Temperature Change,CO₂ Concentration
1961,0.163053,158.938333
1962,-0.013476,159.355000
1963,-0.006043,159.580417
1964,-0.070059,159.905833
1965,-0.247027,160.083333


In [6]:
heatmap_fig = px.imshow(
    merged_data.corr(),
    text_auto=".2f",
    color_continuous_scale="RdBu",
    title="Correlation Heatmap"
)
heatmap_fig.update_layout(
    template="plotly_white"
)
heatmap_fig.show()

In [7]:
scatter_fig = px.scatter(
    merged_data,
    x="CO₂ Concentration", y="Temperature Change",
    labels={"CO₂ Concentration": "CO₂ Concentration (ppm)", "Temperature Change": "Temperature Change (°C)"},
    title="Temperature Change vs CO₂ Concentration",
    template="plotly_white"
  )
scatter_fig.show()

In [8]:
co2_data['Month'] = co2_data['Date'].str[-2:].astype(int)
co2_monthly = co2_data.groupby('Month')['Value'].mean()

fig_seasonal = px.line(
    co2_monthly,
    x=co2_monthly.index,
    y=co2_monthly.values,
    labels={"x": "Month", "y": "CO₂ Concentration (ppm)"},
    title="Seasonal Variations in CO₂ Concentrations",
    markers=True
)
fig_seasonal.show()

In [9]:
from scipy.stats import pearsonr, spearmanr
pearson_corr,_ = pearsonr(merged_data["CO₂ Concentration"], merged_data["Temperature Change"])
spearman_corr,_= spearmanr(merged_data["CO₂ Concentration"], merged_data["Temperature Change"])
print(f"Pearson Correlation: {pearson_corr:.4f}")
print(f"Spearman Correlation: {spearman_corr:.4f}")

Pearson Correlation: 0.9554
Spearman Correlation: 0.9379


In [10]:
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler

In [11]:
clustering_data = merged_data[["Temperature Change", "CO₂ Concentration"]].dropna()

scaler = StandardScaler()
scaled_data = scaler.fit_transform(clustering_data)

kmeans = KMeans(n_clusters=3, random_state=42)
clustering_data['Cluster'] = kmeans.fit_predict(scaled_data)

clustering_data['Label'] = clustering_data['Cluster'].map({
    0: 'Moderate Temp & CO₂',
    1: 'High Temp & CO₂',
    2: 'Low Temp & CO₂'
})


In [12]:
fig_clusters = px.scatter(
    clustering_data,
    x="CO₂ Concentration",
    y="Temperature Change",
    color="Label",
    color_discrete_sequence=px.colors.qualitative.Set2,
    labels={
        "CO₂ Concentration": "CO₂ Concentration (ppm)",
        "Temperature Change": "Temperature Change (°C)",
        "Label": "Climate Pattern"
    },
    title="Clustering of Years Based on Climate Patterns"
)

fig_clusters.update_layout(
    template="plotly_white",
    legend_title="Climate Pattern"
)

fig_clusters.show()

In [16]:
from sklearn.linear_model import LinearRegression

X = merged_data[["CO₂ Concentration"]].values
y = merged_data["Temperature Change"].values

model = LinearRegression()
model.fit(X, y)


def simulate_temperature_change(co2_percentage_change):
    current_mean_co2 = merged_data["CO₂ Concentration"].mean()
    new_co2 = current_mean_co2 * (1 + co2_percentage_change / 100)

    predicted_temp = model.predict([[new_co2]])
    return predicted_temp[0]

print(f"Increase CO₂ by 10%: {simulate_temperature_change(10):.2f} *C\n"
      f"Decrease CO₂ by 10%: {simulate_temperature_change(-10):.2f} *C\n"
      f"Increase CO₂ by 15%: {simulate_temperature_change(15):.2f} *C\n"
      f"Decrease CO₂ by 15%: {simulate_temperature_change(-15):.2f} *C")

Increase CO₂ by 10%: 1.09 *C
Decrease CO₂ by 10%: -0.06 *C
Increase CO₂ by 15%: 1.37 *C
Decrease CO₂ by 15%: -0.35 *C
